In [4]:
#import library
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

from Cleansing import clean

In [18]:
#read data training
trainn = pd.read_csv("dataset/train_preprocess.tsv.txt", sep='\t', names=['Kalimat','Sentiment'])
trainn.head()

,Kalimat,Sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [6]:
#cek data volume
trainn.shape

(11000, 2)

In [7]:
trainn.isna().sum()

Kalimat      0
Sentiment    0
dtype: int64

In [8]:
trainn.Sentiment.value_counts()

positive    6416
negative    3436
neutral     1148
Name: Sentiment, dtype: int64

In [9]:
trainn['Clean'] = trainn.Kalimat.apply(clean)
trainn.head()

,Kalimat,Sentiment,Clean
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung dimiliki pengusaha pabrik puluhan terke...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus k212 mmbri hujjah partai diw...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis jalan sumatra bandung nya nya...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya unboxing paket barang nya b...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,aduh mahasiswa sombong kasih kartu kuning bela...


In [10]:
dataprep = trainn.Clean.to_list()

In [11]:
cv_nn = CountVectorizer()
cv_nn.fit(dataprep)

X = cv_nn.transform(dataprep)
print('Feature Extraction done')
print (X)

Feature Extraction done
  (0, 918)	1
  (0, 1013)	1
  (0, 1163)	1
  (0, 1488)	1
  (0, 1713)	1
  (0, 2098)	1
  (0, 3742)	1
  (0, 3794)	2
  (0, 4086)	1
  (0, 5365)	1
  (0, 5759)	1
  (0, 6297)	1
  (0, 6474)	1
  (0, 7329)	1
  (0, 8398)	1
  (0, 8470)	1
  (0, 9197)	2
  (0, 9228)	1
  (0, 10067)	1
  (0, 10306)	1
  (0, 10998)	1
  (0, 11786)	1
  (0, 11824)	1
  (0, 12861)	1
  (0, 14113)	1
  :	:
  (10999, 911)	1
  (10999, 1168)	1
  (10999, 1224)	1
  (10999, 1383)	1
  (10999, 1485)	1
  (10999, 1867)	1
  (10999, 1999)	1
  (10999, 4020)	1
  (10999, 4763)	1
  (10999, 4764)	1
  (10999, 6212)	1
  (10999, 6312)	1
  (10999, 7406)	1
  (10999, 7735)	1
  (10999, 7873)	1
  (10999, 8026)	1
  (10999, 8027)	2
  (10999, 10067)	3
  (10999, 11410)	1
  (10999, 11411)	1
  (10999, 12207)	1
  (10999, 12243)	1
  (10999, 14653)	1
  (10999, 14809)	1
  (10999, 15531)	1


In [12]:
pickle.dump(cv_nn, open('asset/feature_nn.pickle','wb'))

In [13]:
Y = trainn.Sentiment
Y

0        positive
1         neutral
2        positive
3        positive
4        negative
           ...   
10995    positive
10996    positive
10997     neutral
10998    negative
10999    positive
Name: Sentiment, Length: 11000, dtype: object

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

In [15]:
model_nn = MLPClassifier()
model_nn.fit(X_train, Y_train)

print('Training selesai')

Training selesai


In [16]:
pickle.dump(model_nn, open('asset/model_nn.pickle','wb'))

In [17]:
test = model_nn.predict(X_test)

print ('testing selesai')
print(classification_report(Y_test, test))

testing selesai
              precision    recall  f1-score   support

    negative       0.70      0.75      0.73       717
     neutral       0.68      0.51      0.59       235
    positive       0.84      0.84      0.84      1248

    accuracy                           0.78      2200
   macro avg       0.74      0.70      0.72      2200
weighted avg       0.78      0.78      0.78      2200



In [19]:
# Create a KFold object with 5 splits and a fixed random state
Kf = KFold(n_splits=5, shuffle=True, random_state=42)
akurasi = []

In [20]:
for iteration, data in enumerate(Kf.split(X), start=1):
    data_train = X[data[0]]
    target_train = Y[data[0]]

    data_test = X[data[1]]
    target_test = Y[data[1]]

    clf = MLPClassifier()
    clf.fit(data_train, target_train)
    
    pred = clf.predict(data_test)
    akrsi = accuracy_score(target_test,pred)

    print("Training ke: ", iteration)
    print("---")
    print(classification_report(target_test,pred))

akurasi.append(akrsi)
print("rata-rata akurasi: ", np.mean(akurasi))

Training ke:  1
---
              precision    recall  f1-score   support

    negative       0.69      0.71      0.70       680
     neutral       0.72      0.53      0.61       239
    positive       0.83      0.86      0.85      1281

    accuracy                           0.78      2200
   macro avg       0.75      0.70      0.72      2200
weighted avg       0.78      0.78      0.78      2200

Training ke:  2
---
              precision    recall  f1-score   support

    negative       0.72      0.72      0.72       706
     neutral       0.64      0.57      0.60       220
    positive       0.85      0.86      0.86      1274

    accuracy                           0.79      2200
   macro avg       0.74      0.72      0.73      2200
weighted avg       0.79      0.79      0.79      2200

Training ke:  3
---
              precision    recall  f1-score   support

    negative       0.71      0.72      0.71       682
     neutral       0.73      0.60      0.66       215
    positive   

In [23]:
ori = '''
bacot!!!! kamu terlalu banyak bicara'''

te = cv_nn.transform([clean(ori)])
res = model_nn.predict(te)[0]

print(ori)
print('sentiment:', res)


bacot!!!! kamu terlalu banyak bicara
sentiment: negative
